In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="portfoliovqe",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------portfolio-vqe with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------portfolio-vqe with qubit 3 ---------------
---------------portfolio-vqe with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7dbb5cfd2ab0>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7dbb611145f0>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'101': 42, '110': 4, '010': 42, '011': 5, '111': 7, '001': 21, '100': 879}, {'000': 2.9244413665911116e-05, '001': 0.026979115677541084, '010': 0.04333720432657178, '011': 0.002618824406832032, '100': 0.8917333827453636, '101': 0.029975893278742602, '110': 0.0017130023804962783, '111': 0.0036133327707863635})
(4, {'1010': 155, '1100': 249, '1011': 57, '0111': 14, '0011': 4, '1101': 27, '1000': 3, '0110': 25, '0001': 41, '1110': 33, '1001': 342, '0101': 9, '0100': 25, '0010': 16}, {'0000': 0.0027388870593853696, '0001': 0.044108283866321735, '0010': 0.017149787599059274, '0011': 0.0027013884416906783, '0100': 0.02472336733952661, '0101': 0.009445195008634718, '0110': 0.031540609441247945, '0111': 0.012122680467221748, '1000': 0.0016641233235905415, '1001': 0.33737163101049134, '1010': 0.1431114257004291, '1011': 0.056347085560577354, '1100': 0.2511096723947734, '1101': 0.023765841844088652, '1110': 0.04148459098268075, '1111': 0.0006154299602799933})


In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [4]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (VQE, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (VQE, 3 qubits): 0.9971
Hellinger Fidelity (VQE, 4 qubits): 0.9950
